Projet_Covid

In [1]:
import pandas as pd

url = "https://static.data.gouv.fr/resources/synthese-des-indicateurs-de-suivi-de-lepidemie-covid-19/20230630-155909/table-indicateurs-open-data-dep-2023-06-30-17h59.csv"
df = pd.read_csv(url, sep=";")
df.head()

,"dep,date,reg,lib_dep,lib_reg,tx_pos,tx_incid,TO,R,hosp,rea,rad,dchosp,reg_rea,incid_hosp,incid_rea,incid_rad,incid_dchosp,reg_incid_rea,pos,pos_7j,cv_dose1"
0,"01,2020-03-18,84,Ain,Auvergne et Rhône-Alpes,,..."
1,"01,2020-03-19,84,Ain,Auvergne et Rhône-Alpes,,..."
2,"01,2020-03-20,84,Ain,Auvergne et Rhône-Alpes,,..."
3,"01,2020-03-21,84,Ain,Auvergne et Rhône-Alpes,,..."
4,"01,2020-03-22,84,Ain,Auvergne et Rhône-Alpes,,..."
